# 读入库

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import jiagu
import string
import pandas as pd
import torch
from PIL import Image
import numpy as  np
import cn_clip.clip as clip
from cn_clip.clip import load_from_name, available_models
import itertools
import jiagu
import string
import re
import os
import fitz # PyMuPDF
import pdb
import ast
import math

from tqdm import tqdm

print("Available models:", available_models())
# Available models: ['ViT-B-16', 'ViT-L-14', 'ViT-L-14-336', 'ViT-H-14', 'RN50']

Available models: ['ViT-B-16', 'ViT-L-14', 'ViT-L-14-336', 'ViT-H-14', 'RN50']


# 读入文本和图片信息的DataFrame

In [3]:
# 读入文本DataFrame
df_text = pd.read_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/ESG 2020/01 Text/text_df_2.xlsx")
# df_text = pd.read_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/03_TextImgMatch/partly/文本提取_text_df_2.xlsx")

# 读入图片DataFrame
df_img = pd.read_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/ESG 2020/03 PyMu_img_cord/img_cord.xlsx")
df_img = df_img.drop_duplicates(subset='file_name')

In [4]:
def PDF_name_extract(file_name):
    PDF_name, _, _, _, _ = file_name.split('_')
    return str(PDF_name)

def page_extract(file_name):
    _, _, page, _, _ = file_name.split('_')
    return int(page)

def p_index_extract(file_name):
    _, _, _, _, p_index = file_name.split('_')
    return int(p_index[:-4])

In [5]:
if 'PDF_name' not in df_img.columns:
    df_img['PDF_name'] = df_img['file_name'].apply(PDF_name_extract)
    df_img['page'] = df_img['file_name'].apply(page_extract)
    df_img['p_index'] = df_img['file_name'].apply(p_index_extract)

# 函数定义
## 去除字段中的标点和空格

In [6]:
def remove_punctuation_and_spaces(text):
    # 去除标点符号
    text = str(text)
    text = text.translate(str.maketrans("", "", string.punctuation))

    # 去除空格
    text = re.sub(r"\s+", "", text)

    return text

## 从文本段中提取关键词

In [7]:
# fin = open('GS.txt', mode='r',encoding='UTF-8')
# text = fin.read()
# fin.close()

def labelization(text, key_num = 3):
#     summarize = jiagu.summarize(text, 1) # 摘要
#     print("Sum:", summarize)
#     keywords = jiagu.keywords(text, 15) # 关键词
#     print("\nKeyWords:", keywords)

    cleaned_sentence = remove_punctuation_and_spaces(text)
#     print("\nCleaned Sentence:", cleaned_sentence)
    # summarize1 = jiagu.summarize(cleaned_sentence, 1) # 摘要
    # print(summarize1)
    keywords1 = jiagu.keywords(cleaned_sentence, key_num) # 关键词
#     print("KeyWords:", keywords1,"\n\n")
    
    return keywords1
# fin = open('input.txt', 'r')
# text = fin.read()
# fin.close()
#
# summarize = jiagu.summarize(text, 3) # 摘要
# print(summarize)

## 提取当页全部关键词

In [8]:
def extract_keywords(PDF_name, page):
    keywords_list = []
    keyword = []
#     page = page + 1
    
#     unique_pages = df_text[df_text["PDF_name"] == PDF_name]['page'].unique()
#     for page in unique_pages:
#         # 指定要获取的行
    page_rows = df_text[(df_text["PDF_name"] == PDF_name) & (df_text['page'] == page)]


    for idx, row in page_rows.iterrows():
        # 提取关键词
        if not pd.isna(row['content']):
            keywords = labelization(row['content'], len(str(row['content'])) // 20 + 2)
#         print(keywords)
        # 将关键词保存至特定行的'keyword'列
        df_text['keyword'] = df_text['keyword'].astype('object')
#         df_text.loc[idx, "keyword"] = keywords
        df_text.at[idx, 'keyword'] = keywords

        # 加入当前页的总'keyword_list'
        keywords_list.append(keywords)
        keyword = list(itertools.chain.from_iterable(keywords_list))
    
    return keyword

## 获取特定图片在当页标签中的概率

In [9]:
# img_path = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/Test01/02 PyMu_img01/00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf_page_10_img_1.png"

# keywords = [keyword for keyword in keywords_list if pattern.match(keyword)]

def jiagu_prob(img_path, keywords):
    # 预处理，读入模型
    device='cpu'
    model, preprocess = load_from_name("RN50", device=device, download_root='./')
    model.eval()

    # 转移到mps上进行gpu加速
    device = "mps" if torch.backends.mps.is_available() == True else 'cpu'
    model = model.to(device)

    image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
    label = clip.tokenize(keywords).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(label)
        # 对特征进行归一化，用于下游任务
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        logits_per_image, logits_per_text = model.get_similarity(image, label)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    return probs, img_path

# print("\nLabel probs:", probs)  # [[1.268734e-03 5.436878e-02 6.795761e-04 9.436829e-01]]
# labels = keywords
# max_prob_index = np.argmax(probs)
# max_prob_label = labels[max_prob_index]
# print("\nMax probability label:", max_prob_label)

## 图片匹配文字

In [10]:
def text_match_img(labels, probs, img_path, PDF_name, page, df_text = df_text, df_img = df_img):
    """
    将每个图片最配对文本保留在df_img中
    将最配对文本的可信度（CRI）保留
    """
    # 创建一个标签到概率的映射字典
    prob_dict = dict(zip(labels, probs[0]))
#     print("\nProb_Dict:\n", prob_dict, "\n\n\n")
    filtered_df = df_text[(df_text["PDF_name"] == PDF_name) & (df_text['page'] == page)]
    if not filtered_df.empty:
        filtered_df = filtered_df[pd.notnull(filtered_df['keyword'])]

        # 创建一个新列来存储每个标签组的概率总和
        filtered_df['prob_sum'] = filtered_df['keyword'].apply(
            lambda keywords: sum(prob_dict.get(k, 0) for k in keywords if pd.notnull(k)) / 
                             (len([k for k in keywords if pd.notnull(k)]) if keywords else 1)
        )

        
        # 使用 apply() 方法遍历列
        filtered_df['prob_sum'] = filtered_df['prob_sum'].apply(lambda x: 0.0 if not isinstance(x, float) else x)
#         print("\nFiltered_df:\n",filtered_df, "\n")
        
        if not filtered_df['prob_sum'].empty:
            max_prob_content = filtered_df.loc[filtered_df['prob_sum'].idxmax(), 'content']
            max_prob = filtered_df['prob_sum'].max()
            cri = CRI(filtered_df)
        else:
        # 如果 prob_sum 列为空
            max_prob_content = None  # 或任何合适的默认值
            max_prob = None
            cri = None

        # 获取图片名称并存储至df_img
        img_name = os.path.basename(img_path)
        df_img.loc[df_img["file_name"] == img_name, "match_text"] = max_prob_content
        df_img.loc[df_img["file_name"] == img_name, "match_text_prob"] = max_prob
        df_img.loc[df_img["file_name"] == img_name, "match_text_CRI"] = cri
        
#         print(max_prob_content, max_prob, cri)
        
        return max_prob_content

In [ ]:
# # beta
# def text_match_img(labels, probs, img_path, PDF_name, page, df_text = df_text, df_img = df_img):
#     """
#     将每个图片最配对文本保留在df_img中
#     将最配对文本的可信度（CRI）保留
#     """
#     # 创建一个标签到概率的映射字典
#     prob_dict = dict(zip(labels, probs[0]))
# #     print("\nProb_Dict:\n", prob_dict, "\n\n\n")

#     # 找到物理最近的图文
#     img_name = os.path.basename(img_path)
#     img_cord = df_img[df_img['file_name'] == img_name]['centre_coordinate']
#     [img_cord] = img_cord.values
#     img_cord = ast.literal_eval(img_cord)
    
#     df_min_dis = df_text.loc[(df_text['PDF_name'] == os.path.basename(PDF_path)) & (df_text['page'] == page)]
    
#     if not df_min_dis.empty:
#         df_min_dis['dis_to_img_current'] = df_min_dis.apply(lambda row: distance(img_cord, (row['center_x'], row['center_y'])), axis = 1)
        
#         return df_min_dis.sort_values(by='dis_to_img_current', ascending = True)[0:1]['keyword'].apply(
#                lambda keywords: sum(prob_dict.get(k, 0) for k in keywords if pd.notnull(k)) /
#                          (len([k for k in keywords if pd.notnull(k)]) if keywords else 1)
#             )
#     else:
#         print("{0:=^40}".format('Anomaly Founded'))
#         print("PDF:{} has no text content on page{}".format(PDF_name, page))
#         return -1
    
    

#     # 配对图片与文字
#     filtered_df = df_text[(df_text["PDF_name"] == PDF_name) & (df_text['page'] == page)]
#     if not filtered_df.empty:
#         filtered_df = filtered_df[pd.notnull(filtered_df['keyword'])]

#         # 创建一个新列来存储每个标签组的概率总和
#         filtered_df['prob_sum'] = filtered_df['keyword'].apply(
#             lambda keywords: sum(prob_dict.get(k, 0) for k in keywords if pd.notnull(k)) / 
#                              (len([k for k in keywords if pd.notnull(k)]) if keywords else 1)
#         )

        
#         # 使用 apply() 方法遍历列
#         filtered_df['prob_sum'] = filtered_df['prob_sum'].apply(lambda x: 0.0 if not isinstance(x, float) else x)
# #         print("\nFiltered_df:\n",filtered_df, "\n")
        
#         if not filtered_df['prob_sum'].empty:
#             max_prob_content = filtered_df.loc[filtered_df['prob_sum'].idxmax(), 'content']
#             max_prob = filtered_df['prob_sum'].max()
#             cri = CRI(filtered_df)
#         else:
#         # 如果 prob_sum 列为空
#             max_prob_content = None  # 或任何合适的默认值
#             max_prob = None
#             cri = None

#         # 存储至df_img
#         df_img.loc[df_img["file_name"] == img_name, "match_text"] = max_prob_content
#         df_img.loc[df_img["file_name"] == img_name, "match_text_prob"] = max_prob
#         df_img.loc[df_img["file_name"] == img_name, "match_text_CRI"] = cri
        
#         return max_prob_content

## 计算标签组概率的可靠性

In [11]:
def CRI(filtered_df):
    """
    综合可靠性指数(Composite Reliability Index, CRI)
        计算: 最大概率差异度(Maximum Probability Spread, MPS) / 标准差(Standard Deviation, SD)
    
    最大概率差异度(Maximum Probability Spread, MPS): 
        计算: 最大概率值与其他概率值的差值，然后取平均值。
    """
    # 计算MPS和SD
    max_probability = filtered_df['prob_sum'].max()  # 最大概率值
    average_probability = filtered_df['prob_sum'].mean()  # 所有概率的平均值
    probabilities_except_max = filtered_df[filtered_df['prob_sum'] != max_probability]['prob_sum']  # 除了最大值的所有概率值

    # 计算MPS
    mps = max_probability - probabilities_except_max.mean() if not probabilities_except_max.empty else 0

    # 计算标准差
    sd = filtered_df['prob_sum'].std()  # 所有概率值的标准差

    # 计算CRI
    cri = mps / sd if sd > 0 else 0  # 如果标准差为0，则CRI没有意义
    
    return cri

## 一次获取PDF中每一页的全部标签

In [12]:
# def label_in_PDF(PDF_path):
#     pdf_file = fitz.open(PDF_path)
#     pdf_name = os.path.basename(PDF_path)

#     key = []

#     # 遍历PDF的每一页
#     for page in tqdm(range(len(pdf_file))):
#         keywords = extract_keywords(pdf_name, page)
#     #     print("Page:{}\n{}\n\n".format(page,keywords))
#         key.append({'file_name': pdf_name,
#                     'page': page + 1,
#                     'keywords': keywords})

#     # 创建临时df存放每一页的关键词组 
#     df_key = pd.DataFrame(key)
    
#     return df_key

In [21]:
def label_in_PDF(PDF_path):
    pdf_file = fitz.open(PDF_path)
    pdf_name = os.path.basename(PDF_path)

    key = []

    # 遍历PDF的每一页
    for page in range(len(pdf_file)):
        keywords = extract_keywords(pdf_name, page+1)
    #     print("Page:{}\n{}\n\n".format(page,keywords))
        key.append({'file_name': pdf_name,
                    'page': page + 1,
                    'keywords': keywords})

    # 创建临时df存放每一页的关键词组 
    df_key = pd.DataFrame(key)
    
    return df_key

## 计算距离

In [14]:
def distance(img_cord, text_cord):
    """
    Takes in both 'img_cord' and 'text_cord' in tuple of (x, y).
    Return the distance between two cord.
    """
    img_x, img_y = img_cord
    text_x, text_y = text_cord
    
    return math.sqrt((text_x - img_x)**2 + (text_y - img_y)**2)

In [15]:
def get_bundle_path(directory):
    files = [f for f in os.listdir(directory) if f != '.DS_Store']
    return [os.path.join(directory, file) for file in files if os.path.isfile(os.path.join(directory, file))]

# 计算最近的图文间的语义匹配度

In [16]:
def wording_similarity(labels, probs, img_path, PDF_name, page, df_text = df_text, df_img = df_img):
    # 创建一个标签到概率的映射字典
    prob_dict = dict(zip(labels, probs[0]))
    
    # 获得图片坐标
    img_name = os.path.basename(img_path)
    img_cord = df_img[df_img['file_name'] == img_name]['centre_coordinate']
    [img_cord] = img_cord.values
    img_cord = ast.literal_eval(img_cord)
        
    # 获取文本段坐标
    df_min_dis = df_text.loc[(df_text['PDF_name'] == PDF_name) & (df_text['page'] == page)]
#     df_min_dis = df_min_dis(pd.notnull(df_min_dis['keyword']))
    if not df_min_dis.empty:
        df_min_dis['dis_to_img_current'] = df_min_dis.apply(lambda row: distance(img_cord, (row['center_x'], row['center_y'])), axis = 1)
        
        # 计算标签组概率
        return df_min_dis.sort_values(by='dis_to_img_current', ascending = True)[0:1]['keyword'].apply(
               lambda keywords: sum(prob_dict.get(k, 0) for k in keywords if pd.notnull(k)) /
                         (len([k for k in keywords if pd.notnull(k)]) if keywords else 1)
            )
    else:
        print("{0:=^40}".format('Anomaly Founded'))
        print("PDF:{} has no text content on page{}".format(PDF_name, page))
        return -1

In [19]:
# img_path = '/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/Test01/02 PyMu_img01/00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf_page_10_img_1.png'
# PDF_path = '/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/00_SustainabilityReport_PDF/2022/00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf'
# df_key = label_in_PDF(PDF_path)
# [keywords] = list(df_key.loc[(df_key['file_name'] == os.path.basename(PDF_path)) & (df_key['page'] == 10)]['keywords'])
# [keywords_t1] = list(df_key.loc[(df_key['file_name'] == os.path.basename(PDF_path)) & (df_key['page'] == 8)]['keywords'])

# probs, img_path = jiagu_prob(img_path, keywords)

# text_match_img(keywords, probs, img_path, os.path.basename(PDF_path), page=10)
# ws = wording_similarity(keywords, probs, img_path, os.path.basename(PDF_path), page=10)

# print(ws, type(ws))

100%|███████████████████████████████████████████| 50/50 [00:03<00:00, 13.68it/s]


Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}
2190    0.071095
Name: keyword, dtype: float64 <class 'pandas.core.series.Series'>


/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_5698/2785949079.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_dis['dis_to_img_current'] = df_min_dis.apply(lambda row: distance(img_cord, (row['center_x'], row['center_y'])), axis = 1)


# 运行

In [17]:
# 指定要处理的PDF路径
# PDF_path = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/00_SustainabilityReport_PDF/2022/00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf"

In [18]:
# 指定要处理的PDF文件夹
# folder_dir = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/00_SustainabilityReport_PDF/2022"
folder_dir = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2020"

# 指定要处理的图片文件夹
img_folder = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/ESG 2020/02 PyMu_img"

# 已经处理过的df_distance
result_path = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/03_TextImgMatch/distance/ESG_2020_distance.xlsx"

In [24]:
def main(batch_size = 20, folder_dir = folder_dir, df_img = df_img,
        img_folder = img_folder,
        result_path = result_path,
        output_path = '/Users/improvise/Desktop/保研/实证论文/ESG/Playground/03_TextImgMatch/distance'):
    
    # 移除已经运行过的PDF
    if result_path != None:
        df_dis = pd.read_excel(result_path)
        file_mask = df_dis['PDF_name'].unique().tolist()
        df_img = df_img[~df_img['PDF_name'].isin(file_mask)]
    
    print("\n{} PDF files remaining.\n".format(len(df_img['PDF_name'].unique().tolist())))
    # 取出一个batch_size
    pdf_list = df_img['PDF_name'].unique().tolist()[0:batch_size]
    PDF_path_list = get_bundle_path(folder_dir)
    PDF_path_list = [i for i in PDF_path_list if os.path.basename(i) in pdf_list]
#     print(PDF_path_list)
    
    # 运行
    for PDF_path in tqdm(PDF_path_list):
        df_key = label_in_PDF(PDF_path)
        for row in df_img.loc[df_img['PDF_name']==os.path.basename(PDF_path)].itertuples():
            img_name = row.file_name
            page = row.page
            p_index = row.p_index
            img_cord = row.centre_coordinate
    #         print('\n\n', page, p_index, '\n\n')

            # 读图
            img_path = os.path.join(img_folder, img_name)
            # 读页面关键词
            [keywords] = list(df_key.loc[(df_key['file_name'] == os.path.basename(PDF_path)) & (df_key['page'] == page)]['keywords'])
    #         print('Keywords:\n', keywords,'\n')
    #         print('Keywords_t1:\n', keywords_t1, '\n\n\n')

            # 计算概率
#             ipdb.set_trace()
            probs, img_path = jiagu_prob(img_path, keywords)

            # 计算语义近似度
            ws = wording_similarity(keywords, probs, img_path, os.path.basename(PDF_path), page=page, df_img = df_img)
            if isinstance(ws, pd.Series):
                [ws] = list(ws) 
            df_img.loc[df_img["file_name"] == img_name, "word_simi"] = ws
#             print("\n\nWording Similarity:\n", ws, "\n")

            # 匹配图文
            object_text = text_match_img(keywords, probs, img_path, os.path.basename(PDF_path), page, df_img = df_img)
            if object_text != None:
            # 获取文本坐标
                text_cord_list = df_text.loc[(df_text['PDF_name'] == os.path.basename(PDF_path)) & (df_text['page'] == page) & (df_text['content'] == object_text)][['center_x','center_y']].values.tolist()

                if text_cord_list:
                    text_cord = text_cord_list[0]  # 选择第一个元素

                # 接下来的计算
                else:
                    [text_cord] = text_cord_list

                # 计算距离
                if type(img_cord) == str:
                        img_cord = ast.literal_eval(img_cord)

                dist = distance(img_cord, text_cord)
                df_img.loc[df_img["file_name"] == img_name, "distance"] = dist
    
    # 将运行结果与上次的运行结果合并
    df_img = df_img[~df_img['PDF_name'].isin(df_img['PDF_name'].unique().tolist()[batch_size:])]
    if result_path != None:
        df_img_2 = pd.concat([df_dis ,df_img], ignore_index=True)
        df_img_2.to_excel(os.path.join(output_path, 'ESG_2020_distance.xlsx'), index = False)
    else:
        df_img.to_excel(os.path.join(output_path, 'ESG_2020_distance.xlsx'), index = Falses)
        
    print("Saved as {} at {}".format('ESG_2020_distance.xlsx', output_path))

In [25]:
main(batch_size=2, result_path="/Users/improvise/Desktop/保研/实证论文/ESG/Playground/03_TextImgMatch/distance/ESG_2020_distance.xlsx")


23 PDF files remaining.




  0%|                                                     | 0/2 [00:00<?, ?it/s]

============Anomaly Founded=============
PDF:600660.SH-福耀玻璃-福耀玻璃2020年环境、社会及管治报告-2021-04-16.pdf has no text content on page1
============Anomaly Founded=============
PDF:600660.SH-福耀玻璃-福耀玻璃2020年环境、社会及管治报告-2021-04-16.pdf has no text content on page2
============Anomaly Founded=============
PDF:600660.SH-福耀玻璃-福耀玻璃2020年环境、社会及管治报告-2021-04-16.pdf has no text content on page9
============Anomaly Founded=============
PDF:600660.SH-福耀玻璃-福耀玻璃2020年环境、社会及管治报告-2021-04-16.pdf has no text content on page12
============Anomaly Founded=============
PDF:600660.SH-福耀玻璃-福耀玻璃2020年环境、社会及管治报告-2021-04-16.pdf has no text content on page17
============Anomaly Founded=============
PDF:600660.SH-福耀玻璃-福耀玻璃2020年环境、社会及管治报告-2021-04-16.pdf has no text content on page24
============Anomaly Founded=============
PDF:600660.SH-福耀玻璃-福耀玻璃2020年环境、社会及管治报告-2021-04-16.pdf has no text content on page32



 50%|██████████████████████                      | 1/2 [02:26<02:26, 146.85s/it]

============Anomaly Founded=============
PDF:600660.SH-福耀玻璃-福耀玻璃2020年环境、社会及管治报告-2021-04-16.pdf has no text content on page40


100%|████████████████████████████████████████████| 2/2 [06:27<00:00, 193.70s/it]


Saved as ESG_2020_distance.xlsx at /Users/improvise/Desktop/保研/实证论文/ESG/Playground/03_TextImgMatch/distance


In [19]:
# PDF_path_list = get_bundle_path(folder_dir)

# for PDF_path in PDF_path_list:
#     df_key = label_in_PDF(PDF_path)
#     for row in df_img.loc[df_img['PDF_name']==os.path.basename(PDF_path)].itertuples():
#         img_name = row.file_name
#         page = row.page
#         p_index = row.p_index
#         img_cord = row.centre_coordinate
# #         print('\n\n', page, p_index, '\n\n')
        
#         # 读图
#         img_path = os.path.join(img_folder, img_name)
#         # 读页面关键词
#         [keywords] = list(df_key.loc[(df_key['file_name'] == os.path.basename(PDF_path)) & (df_key['page'] == page)]['keywords'])
# #         print('Keywords:\n', keywords,'\n')
# #         print('Keywords_t1:\n', keywords_t1, '\n\n\n')
        
#         # 计算概率
#         probs, img_path = jiagu_prob(img_path, keywords)

#         # 计算语义近似度
#         ws = wording_similarity(keywords, probs, img_path, os.path.basename(PDF_path), page=page)
#         if isinstance(ws, pd.Series):
#             [ws] = list(ws) 
#         df_img.loc[df_img["file_name"] == img_name, "word_simi"] = ws
#         print("\n\nWording Similarity:\n", ws, "\n")
        
#         # 匹配图文
#         object_text = text_match_img(keywords, probs, img_path, os.path.basename(PDF_path), page)
#         if object_text != None:
#         # 获取文本坐标
#             text_cord_list = df_text.loc[(df_text['PDF_name'] == os.path.basename(PDF_path)) & (df_text['page'] == page) & (df_text['content'] == object_text)][['center_x','center_y']].values.tolist()

#             if text_cord_list:
#                 text_cord = text_cord_list[0]  # 选择第一个元素

#             # 接下来的计算
#             else:
#                 [text_cord] = text_cord_list

#             # 计算距离
#             if type(img_cord) == str:
#                     img_cord = ast.literal_eval(img_cord)

#             dist = distance(img_cord, text_cord)
#             df_img.loc[df_img["file_name"] == img_name, "distance"] = dist

100%|██████████████████████████████████████████| 77/77 [00:00<00:00, 122.62it/s]


Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}
============Anomaly Founded=============
PDF:601919.SH-中远海控-中远海控2020年可持续发展报告-2021-03-31.pdf has no text content on page1


Wording Similarity:
 -1 

Loading vision model config from /Users/improvise/miniconda/en

/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_29292/4260134725.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_dis['dis_to_img_current'] = df_min_dis.apply(lambda row: distance(img_cord, (row['center_x'], row['center_y'])), axis = 1)




Wording Similarity:
 0.02248610332454144 

Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}


/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_29292/4260134725.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_dis['dis_to_img_current'] = df_min_dis.apply(lambda row: distance(img_cord, (row['center_x'], row['center_y'])), axis = 1)


...

============Anomaly Founded=============
PDF:600872.SH-中炬高新-中炬高新2020年年度社会责任暨可持续发展报告-2021-03-24.pdf has no text content on page32


Wording Similarity:
 -1 

Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}
============Anomaly Founded=============
PDF:600872.SH

============Anomaly Founded=============
PDF:600872.SH-中炬高新-中炬高新2020年年度社会责任暨可持续发展报告-2021-03-24.pdf has no text content on page35


Wording Similarity:
 -1 

Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}
============Anomaly Founded=============
PDF:600872.SH

============Anomaly Founded=============
PDF:600872.SH-中炬高新-中炬高新2020年年度社会责任暨可持续发展报告-2021-03-24.pdf has no text content on page35


Wording Similarity:
 -1 

Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}
============Anomaly Founded=============
PDF:600872.SH

============Anomaly Founded=============
PDF:600872.SH-中炬高新-中炬高新2020年年度社会责任暨可持续发展报告-2021-03-24.pdf has no text content on page37


Wording Similarity:
 -1 

Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}
============Anomaly Founded=============
PDF:600872.SH

100%|███████████████████████████████████████████| 42/42 [00:00<00:00, 94.58it/s]


In [21]:
# df_img.to_excel("/Users/improvise/Downloads/SUS_2020_distance.xlsx", index=False)

> Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}

上面的信息被注释了，见
```Python
> /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/utils.py(102)load_from_name()

    101 
--> 102     model = create_model(model_name, checkpoint)
    103     if str(device) == "cpu":
    
> /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/utils.py  
    
    def create_model(model_name, checkpoint=None):
        vision_model, text_model = model_name.split('@')
        # Initialize the model.
        vision_model_config_file = Path(
            __file__).parent / f"model_configs/{vision_model.replace('/', '-')}.json"
    --> # print('Loading vision model config from', vision_model_config_file)
        assert os.path.exists(vision_model_config_file)

        text_model_config_file = Path(
            __file__).parent / f"model_configs/{text_model.replace('/', '-')}.json"
    --> # print('Loading text model config from', text_model_config_file)
        assert os.path.exists(text_model_config_file)

        with open(vision_model_config_file, 'r') as fv, open(text_model_config_file, 'r') as ft:
            model_info = json.load(fv)
            for k, v in json.load(ft).items():
                model_info[k] = v
        if isinstance(model_info['vision_layers'], str):
            model_info['vision_layers'] = eval(model_info['vision_layers'])
    --> # print('Model info', model_info)
        model = CLIP(**model_info)
        convert_weights(model)
        if checkpoint:
            sd = checkpoint["state_dict"]
            if next(iter(sd.items()))[0].startswith('module'):
                sd = {k[len('module.'):]: v for k, v in sd.items() if "bert.pooler" not in k}
            model.load_state_dict(sd)
        return model
```
